In [1]:
import math
import copy

import compound_freecad as FC_Cmp
import core_freecad_ops as FC_Ops
import core_freecad_commands as FC_Cmd

import compound_cameo as MD_Cmp
import core_cameo_ops as MD_Ops
import core_cameo_commands as MD_Cmd

import recipe_house_floor as RHF
import recipe_house_ext_wall as RHEW

Functions to start the editing session and to end it.

# Cameo Functions

Functions for linking to pre-existing data.

## FreeCAD Modify CAD objects

In [2]:
def freecad_remove_object(item='', old_owner=''):
    return {
        'action': 'remove',
        'item': item,
        'old owner': old_owner
    }

In [3]:
def freecad_add_object(item='', new_owner=''):
    return {
        'action': 'add',
        'item': item,
        'new owner': new_owner
    }

In [4]:
def freecad_adjust_links(obj='', new_parent=''):
    return {
        'action': 'move link',
        'object': obj,
        'target': new_parent
    }

# Flooring Frame example

Apply the recipe to an example floor frame.

In [5]:
base_frame = RHF.T_Housing_Flooring_Frame(0)
base_frame.name = "Floor Frame"
base_frame.base_depth = 12.0*42.0
base_frame.base_width = 12.0*32.0
[base_frame.base_depth, base_frame.base_width]

[512.0, 384.0]

In [6]:
girder_board = RHF.T_Housing_Board(0)
girder_board.name = 'Girder'
girder_board.width = 5.5
girder_board.thickness = 1.5
girder_board.length = base_frame.base_depth

In [7]:
sill_board = RHF.T_Housing_Board(0)
sill_board.name = 'Sill Board'
sill_board.width = 5.5
sill_board.thickness = 9.25
sill_board.length = base_frame.base_depth

In [8]:
joist_board = RHF.T_Housing_Board(0)
joist_board.name = 'Joist'
joist_board.width = 3.5
joist_board.thickness = 1.5
joist_board.length = base_frame.base_width - 2.0 * joist_board.thickness

In [9]:
header_joist_board = RHF.T_Housing_Board(0)
header_joist_board.name = 'Header Joist'
header_joist_board.width = 3.5
header_joist_board.thickness = 1.5
header_joist_board.length = base_frame.base_depth

In [10]:
floor_across = 8
floor_down = 8
floor_board = RHF.T_Housing_Panel(0)
floor_board.name = 'Floor Board'
floor_board.width = base_frame.base_width / float(floor_across)
floor_board.depth = base_frame.base_depth / float(floor_down)
floor_board.thickness = 0.25

[floor_board.width, floor_board.depth]

[48.0, 64.0]

In [11]:
base_frame.sill_board_kind = sill_board
base_frame.girder_board_kind = girder_board
base_frame.joist_board_kind = joist_board
base_frame.floor_board_kind = floor_board
base_frame.header_joist_board_kind = header_joist_board

base_frame.panels_across = floor_across
base_frame.panels_down = floor_down

Actually build the example. First the Cameo portion.

In [12]:
build_list = []

In [13]:
build_list = MD_Cmd.access_libraries()

In [14]:
build_list.extend(MD_Cmd.cameo_build_session_start())

In [15]:
stringified = []
for dikt in MD_Cmp.cameo_package_create_dicts(
        name="Base Package", order_start=1, owner_obj='modelBase'):
    stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
build_list.extend(stringified)

In [16]:
stringified = []
for dikt in base_frame.generate_for_cameo_abstract('pkg_id_1'):
    stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
    
build_list.extend(stringified)

In [17]:
build_list.extend(MD_Cmd.cameo_build_session_end())

In [18]:
f = open("cameo_instructions.txt", "w")
for build in build_list:
    f.write(build + '\n')
f.close()

Now build the corresponding FreeCAD model.

In [20]:
cad_stringified = []

cad_stringified.extend(FC_Cmd.freecad_build_session_start())

for dikt in base_frame.generate_for_freecad():
    cad_stringified.append(FC_Cmd.freecad_dict_to_string_router(dikt))
    
cad_stringified.extend(FC_Cmd.freecad_build_session_end())
    
f = open("generate_" + "Floor Frame".replace(" ", "_") + ".py", "w")

for build in cad_stringified:
    f.write(build + '\n')
f.close()

In [21]:
wall_frame = RHEW.T_Housing_Wall_Frame(0)
wall_frame.name = "Wall Frame"
wall_frame.base_length = base_frame.base_depth
wall_frame.base_height = 12.0*10.5
wall_frame.direction = 'North-South'
[wall_frame.base_length, wall_frame.base_height]

[512.0, 126.0]

In [22]:
plate_board = RHF.T_Housing_Board(0)
plate_board.name = 'Plate'
plate_board.width = 1.5
plate_board.thickness = 5.5
plate_board.length = base_frame.base_depth

In [23]:
stud_board = RHF.T_Housing_Board(0)
stud_board.name = 'Stud'
stud_board.width = 1.5
stud_board.thickness = 5.5
stud_board.length = wall_frame.base_height - 2.0 * plate_board.thickness

In [24]:
door_hole = RHEW.T_Housing_Hole()
door_hole.width = 3.0 * 12.0
door_hole.height_from_floor = 0.0
door_hole.height = 8.0 * 12.0
door_hole.position = wall_frame.base_length / 2.0

wall_frame.holes.append(door_hole)

In [25]:
window_hole = RHEW.T_Housing_Hole()
window_hole.width = 6.0 * 12.0
window_hole.height_from_floor = 2.5 * 12.0
window_hole.height = 4.0 * 12.0
window_hole.position = 24.0

wall_frame.holes.append(window_hole)

In [26]:
wall_frame.stud_board_kind = stud_board
wall_frame.plate_board_kind = plate_board

Translate frame based on floor frame

In [27]:
wall_frame.push_x = -base_frame.base_width / 2.0
wall_frame.push_z = base_frame.girder_board_kind.width / 2.0 + \
                        base_frame.joist_board_kind.width + base_frame.floor_board_kind.thickness + \
                        wall_frame.plate_board_kind.width / 2.0

In [28]:
cad_stringified = []

cad_stringified.extend(FC_Cmd.freecad_build_session_start())

for dikt in wall_frame.generate_for_freecad():
    cad_stringified.append(FC_Cmd.freecad_dict_to_string_router(dikt))
    
cad_stringified.extend(FC_Cmd.freecad_build_session_end())
    
f = open("generate_" + "Wall Frame".replace(" ", "_") + ".py", "w")

for build in cad_stringified:
    f.write(build + '\n')
f.close()

In [29]:
cad_stringified = []

cad_stringified.extend(FC_Cmd.freecad_build_session_start())

for dikt in base_frame.generate_for_freecad():
    cad_stringified.append(FC_Cmd.freecad_dict_to_string_router(dikt))
    
indx_push = base_frame.final_indx
wall_frame.push_indx = indx_push

for dikt in wall_frame.generate_for_freecad():
    cad_stringified.append(FC_Cmd.freecad_dict_to_string_router(dikt))
    
cad_stringified.extend(FC_Cmd.freecad_build_session_end())
    
f = open("generate_" + "Builder Frame".replace(" ", "_") + ".py", "w")

for build in cad_stringified:
    f.write(build + '\n')
f.close()